In [ ]:
# opengrid imports
from opengrid_dev.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid_dev import config
c=config.Config()

# other imports
import pandas as pd
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

In [ ]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='water') # sensor objects

hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
# The first time, this will take a very looong time to get all the detailed data for building the cache
# Afterwards, this is quick
caching.cache_results(hp=hp, sensors=sensors, resultname='water_mean', AnalysisClass=DailyAgg,  
                      agg='sum')

In [ ]:
cache_water = caching.Cache(variable='water_mean')
dfday = cache_water.get(sensors=sensors)

In [ ]:
# remove negatives
dfday = dfday[dfday>0]
dfday = dfday.resample('M', how='sum', label='left' )
dfday.info()

In [ ]:
charts.plot(dfday, stock=True, show='inline')

In [ ]:
standby_statistics = dfday.T.describe(percentiles=[0.1,0.5,0.9]).T

In [ ]:
charts.plot(standby_statistics[['10%', '50%', '90%']], stock=True, show='inline')

### Percentile approach.  Useful for longer time periods, but tweaking of graph still needed

In [ ]:
# choose a period
now = pd.Timestamp('now', tz=BXL)
dt_start_of_last_day = pd.Timestamp(dfday.index[-1].date(), tz=BXL)
look_back_days = 400
dt_start_of_period = dt_start_of_last_day - pd.Timedelta(days=look_back_days-1)
dfday_period = dfday.ix[dt_start_of_period:].dropna(axis=1, how='all')
df = dfday_period.join(standby_statistics[['10%', '50%', '90%']], how='left')
df = df.ix[:-1]

In [ ]:
for sensor in dfday_period.columns:
    try:
        plt.figure(figsize=(10,8))
        ax1=plt.subplot()
        ax1.plot_date(df.index, df[u'10%'], '-', lw=2, color='g', label=u'10% percentile')
        ax1.plot_date(df.index, df[u'50%'], '-', lw=2, color='orange', label=u'50% percentile')
        ax1.plot_date(df.index, df[u'90%'], '-', lw=2, color='r', label=u'90% percentile')
        ax1.plot_date(df.index, df[sensor], 'rD', ms=7, label='Your water consumption') 
        ax1.legend(loc='upper center', ncol=4, prop={'size':10})
        locs, lables=plt.xticks()
        xticks = [x.strftime(format='%b') for x in num2date(locs)]
        plt.xticks(locs, xticks, rotation='vertical')
        plt.title(hp.find_sensor(sensor).device.key + ' - ' + sensor)
        ax1.grid()
        ax1.set_ylabel('Liter')

    
    except Exception as e:
        print(e)
    else:
        plt.savefig(os.path.join(c.get('data', 'folder'), 'figures', 'daily_water_'+sensor+'.png'), dpi=100)
        pass
    if not DEV:
        plt.close()